# SBTi-Finance Tool - What-if Analysis
In this notebook we show you how to analyse what impact different actions would have on the temperature score of your portfolio.

Please see the [methodology, guidance](https://sciencebasedtargets.org/wp-content/uploads/2020/07/Temperature-Scoring-Beta-Methodology.pdf) and the [technical documentation](http://getting-started.sbti-tool.org/) for more details on the different what-if scenarios.

See 1_analysis_example for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular. 


## Setting up
First we will set up the imports, data providers, and load the portfolio. 

For more examples of this process, please refer to notebook 1 & 2 (analysis and quick calculation example).

In [1]:
# Only required if you didn't install the module yet. Uncomment the second line and comment out the third line if you wish to use the official release.
# !pip install SBTi
!pip install git+git://github.com/OFBDABV/SBTi

Looking in indexes: http://rdnexus1.ortec.finance:8081/nexus/repository/pypi-all/simple


In [1]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
%aimport -pandas
import pandas as pd

In [2]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/example_portfolio.csv", "data/example_portfolio.csv")

In [3]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")
df_portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1")
companies = SBTi.utils.dataframe_to_portfolio(df_portfolio)
portfolio_data = SBTi.utils.get_data([provider], companies)
scenarios = {}

## Base scenario
To determine what the effect is of running a certain scenario, we first need a base line. To do so, first we calculate the portfolio temperature score, without defining a scenario. 

We can then analyse the effect of the actions in our scenario's by comparing the temperature score medium-term timeframe in the combined s1s2s3 scope. Of course you can use multiple, or different criteria and also change the scope used to calculate the scores.

In [4]:
time_frames = list(SBTi.interfaces.ETimeFrames) 
scopes = [EScope.S1S2, EScope.S3, EScope.S1S2S3]

Below we compute the temperature scores for all the timeframe and scope combinations. Subsequently, one can query the score of interest as shown in the last line in the cell below.

In [5]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes)
base_scenario = temperature_score.calculate(portfolio_data.copy())
base_scenario_aggregated = temperature_score.aggregate_scores(base_scenario)
base_scenario_aggregated.mid.S1S2S3.all.score

2.8756363636363633

## Scenario 1
In scenario 1 we engage companies to set emissions reduction targets of at least 2 degrees Celsius. 

In practice, in the model, this means that we change the default score (i.e. the score when a company didn't set a target) to 2 degrees Celcius.

In [20]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.TARGETS
scenario.engagement_type = None
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [21]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_one = temperature_score.calculate(portfolio_data.copy())
base_scenario_aggregated = temperature_score.aggregate_scores(base_scenario)
base_score = base_scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Base Scenario': base_score})

## Scenario 2
In scenario 2 we engage with companies to influence them to set SBTi approved emissions reduction targets. This means that the maximum score for all targets will be capped at 1.75 degrees Celcius.

In [22]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.APPROVED_TARGETS
scenario.engagement_type = None
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [24]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_2_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 2': scenario_2_score})
print(scenario_2_score)


1.6257272727272722


## Scenario 3a
In scenario 3a we engage the top 10 contributing companies to the portfolio temperature score to influence them to set emission reduction targets. This means that their default score (i.e. the score when a company didn't set a target) will be set to 2 degrees celcius.

In [25]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS
scenario.engagement_type = EngagementType.SET_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [26]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_3a_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 3a': scenario_3a_score})
print(scenario_3a_score)

2.5413636363636365


## Scenario 3b
In scenario 3b we engage the top 10 contributing companies to the portfolio temperature score to influence them to set SBTi-approved targets. This means that the maximum score for all the targets of these companies will be capped at 1.75 degrees celcius.

In [27]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS
scenario.engagement_type = EngagementType.SET_SBTI_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [28]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_3b_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 3b': scenario_3b_score})
print(scenario_3b_score)

2.4735454545454543


## Scenario 4a
In scenario 4a we engage with any company in the portfolio to influence them to set emission reduction targets. We define these companies by adding "`TRUE`" in the `engagement_target`-column in the portfolio input-file. 

This means that their default score (i.e. the score when a company didn't set a target) will be set to 2 degrees celcius.

Scenario 4a and 4b gives you a lot of flexibility, as you can define the companies with which you engage freely and for example focus your engagement efforts on certain sectors, countries, market cap buckets to ensure that you spend your time wisely. 

In [29]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS_APPROVED
scenario.engagement_type = EngagementType.SET_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [30]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_4a_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 4a': scenario_4a_score})
print(scenario_4a_score)

2.7231818181818177


## Scenario 4b
In scenario 4b we can engage with any company in the portfolio and manage to influence them to set SBTi-approved targets. We define these companies by adding "`TRUE`" in the `engagement_target`-column in the portfolio input-file. 

This means that the maximum score for all the targets of these companies will be capped at 1.75 degrees celcius.

In [31]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS_APPROVED
scenario.engagement_type = EngagementType.SET_SBTI_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [32]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_4b_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 4b': scenario_4b_score})
print(scenario_4b_score)

2.690909090909091


To see which scenario provides the most suitable outcome we print an overview of all scenarios:

In [33]:
print('Scenario overview:')
for scenario, score in scenarios.items():
    print('{} \t {:.2f}'.format(scenario, score))

Scenario overview:
Base Scenario 	 2.88
Scenario 2 	 1.63
Scenario 3a 	 2.54
Scenario 3b 	 2.47
Scenario 4a 	 2.72
Scenario 4b 	 2.69
